In [1]:
import pandas as pd

off = pd.read_csv('./offensive_stats.csv')
off = off[['Player', 'FantPt', 'FantPos']]
off.rename(columns={"FantPt":"fpts"}, inplace=True)

rbs = off[off['FantPos'] == "RB"]
wrs = off[off['FantPos'] == "WR"]
tes = off[off['FantPos'] == "TE"]
qbs = off[off['FantPos'] == "QB"]

def_players = pd.read_csv('./stats.csv')
def_players['return_yards'] = def_players['Yds'] + def_players['Yds.1']
def_players['tds'] = def_players['TD'] + def_players['TD.1']
def_players.drop(['Yds', 'TD', 'Yds.1', 'TD.1', 'Age', 'Rk'], axis='columns', inplace=True)
def_players['Sfty'].fillna(0, inplace=True)
def_players['Player'] = def_players['Player'].apply(lambda x : x.split('\\')[0])
def_players.head()


def clean_position(position):
    last_two_letters = position[-2:len(position)].upper()
    if last_two_letters == "LB":
        return "LB"
    elif last_two_letters in ["SS", "CB", "FS", "DB"]:
        return "DB"
    elif last_two_letters in ["DE", "DT", "NT"]:
        return "DL"
    else:
        return position

def_players['Pos'] = def_players['Pos'].astype(str)    
def_players['Pos'] = def_players['Pos'].apply(clean_position)
def_players = def_players[def_players['Pos'] != "nan"]
def_players['Pos'].value_counts()
# DEF stats are Solo, Ast, TFL, PD, Int, Sk, FF, FR, return_yards, tds, 
scoring_system = {
    "Solo": 1,
    "TFL": 3,
    "PD": 3,
    "Int": 6,
    "Sk": 6,
    "FF": 3,
    "FR": 3,
    "return_yards": .1,
    "tds": 6
}

def calculate_fpts(player):
    total = 0
    for key in scoring_system:
        total += player[key] * scoring_system[key]
    return total

def_players['fpts'] = def_players.apply(calculate_fpts, axis=1)
lbs = def_players[def_players['Pos'] == "LB"]
dbs = def_players[def_players['Pos'] == "DB"]
dls = def_players[def_players['Pos'] == "DL"]

positions = [lbs, dbs, dls, rbs, wrs, tes, qbs]
position_names = ["LB", "DB", "DL", "RB", "WR", "TE", "QB"]
ultimate_data = []
index = 0
for position in positions:
    nums = position.sort_values(by="fpts", ascending=False).head(72)["fpts"].describe().to_dict()
    nums['position'] = position_names[index]
    index += 1
    ultimate_data.append(nums)
ultimate_df = pd.DataFrame(ultimate_data)
ultimate_df.set_index("position", inplace=True)
ultimate_df.head(10)

,count,mean,std,min,25%,50%,75%,max
position,,,,,,,,
LB,72.0,114.268056,44.157005,47.0,78.125,109.5,146.550,235.0
DB,72.0,111.211111,28.054356,74.0,85.950,107.0,131.075,173.7
DL,61.0,87.485246,38.962759,30.0,54.000,78.4,124.000,168.0
RB,72.0,116.638889,60.232017,46.0,69.000,108.5,147.250,314.0
WR,72.0,123.152778,39.921987,69.0,94.500,115.0,145.250,243.0
TE,50.0,67.460000,36.988807,30.0,40.250,53.5,86.750,208.0
QB,44.0,204.454545,112.343836,30.0,114.000,189.5,288.000,395.0


In [178]:
# Sanity check to ensure top names are top scorers
lbs.sort_values(by="fpts", ascending=False).head(36)

,Player,Tm,Pos,G,GS,Int,Lng,PD,FF,Fmb,...,Sk,Comb,Solo,Ast,TFL,QBHits,Sfty,return_yards,tds,fpts
161,T.J. Watt *+,PIT,LB,15,15,1.0,0.0,7.0,2.0,0.0,...,15.0,53.0,43.0,10.0,23.0,41.0,0.0,0.0,0.0,235.0
1,Roquan Smith,CHI,LB,16,16,2.0,11.0,7.0,1.0,1.0,...,4.0,139.0,98.0,41.0,18.0,6.0,0.0,16.0,0.0,216.6
2,Devin White,TAM,LB,15,15,0.0,0.0,4.0,1.0,0.0,...,9.0,140.0,97.0,43.0,15.0,16.0,0.0,2.0,0.0,214.2
159,Haason Reddick,ARI,LB,16,11,0.0,0.0,4.0,6.0,0.0,...,12.5,63.0,43.0,20.0,15.0,16.0,0.0,0.0,0.0,193.0
26,Deion Jones,ATL,LB,16,16,2.0,67.0,6.0,1.0,0.0,...,4.5,106.0,73.0,33.0,9.0,10.0,0.0,84.0,1.0,177.4
214,Za'Darius Smith*,GNB,LB,16,16,0.0,0.0,2.0,4.0,0.0,...,12.5,52.0,35.0,17.0,12.0,23.0,0.0,0.0,0.0,170.0
11,Lavonte David,TAM,LB,16,16,1.0,-5.0,6.0,3.0,0.0,...,1.5,117.0,82.0,35.0,12.0,3.0,0.0,-5.0,0.0,165.5
10,Joe Schobert,JAX,LB,16,16,3.0,43.0,4.0,2.0,0.0,...,2.5,141.0,84.0,57.0,6.0,6.0,0.0,45.0,1.0,163.5
224,Jason Pierre-Paul*,TAM,LB,16,16,2.0,15.0,6.0,4.0,0.0,...,9.5,55.0,34.0,21.0,7.0,14.0,0.0,21.0,0.0,162.1
8,Darius Leonard*+,IND,LB,14,14,0.0,0.0,7.0,3.0,1.0,...,3.0,132.0,86.0,46.0,7.0,3.0,0.0,3.0,0.0,161.3


In [179]:
dbs.sort_values(by="fpts", ascending=False).head(36)

,Player,Tm,Pos,G,GS,Int,Lng,PD,FF,Fmb,...,Sk,Comb,Solo,Ast,TFL,QBHits,Sfty,return_yards,tds,fpts
170,Xavien Howard*+,MIA,DB,16,16,10.0,29.0,20.0,1.0,0.0,...,0.0,51.0,40.0,11.0,1.0,0.0,0.0,77.0,0.0,173.7
37,Kenny Moore,IND,DB,16,14,4.0,29.0,13.0,1.0,0.0,...,2.0,80.0,68.0,12.0,4.0,3.0,0.0,45.0,1.0,168.5
5,Budda Baker *+,ARI,DB,15,15,2.0,90.0,6.0,1.0,0.0,...,2.0,118.0,90.0,28.0,7.0,3.0,0.0,90.0,0.0,165.0
34,Malcolm Jenkins,NOR,DB,16,16,3.0,48.0,10.0,1.0,0.0,...,2.5,91.0,69.0,22.0,7.0,6.0,0.0,58.0,0.0,164.8
70,Jamal Adams *,SEA,DB,12,12,0.0,0.0,3.0,1.0,0.0,...,9.5,83.0,59.0,24.0,11.0,14.0,0.0,0.0,0.0,161.0
31,Marlon Humphrey *,BAL,DB,15,15,1.0,8.0,11.0,8.0,0.0,...,2.5,82.0,70.0,12.0,4.0,4.0,1.0,8.0,0.0,160.8
19,Justin Simmons *,DEN,DB,16,16,5.0,46.0,9.0,0.0,0.0,...,0.0,96.0,77.0,19.0,3.0,0.0,0.0,108.0,0.0,156.8
15,Jessie Bates III,CIN,DB,16,16,3.0,12.0,15.0,1.0,0.0,...,0.0,109.0,78.0,31.0,2.0,1.0,0.0,12.0,0.0,151.2
219,J.C. Jackson,NWE,DB,16,11,9.0,30.0,14.0,0.0,0.0,...,0.0,40.0,34.0,6.0,1.0,0.0,0.0,80.0,0.0,150.0
4,Jordan Poyer,BUF,DB,16,16,2.0,14.0,5.0,2.0,0.0,...,2.0,124.0,91.0,33.0,4.0,4.0,0.0,14.0,0.0,149.4


In [180]:
dls.sort_values(by="fpts", ascending=False).head(36)

,Player,Tm,Pos,G,GS,Int,Lng,PD,FF,Fmb,...,Sk,Comb,Solo,Ast,TFL,QBHits,Sfty,return_yards,tds,fpts
302,Aaron Donald*+,LAR,DL,16,16,0.0,0.0,1.0,4.0,0.0,...,13.5,45.0,27.0,18.0,14.0,28.0,0.0,0.0,0.0,168.0
232,Myles Garrett *+,CLE,DL,14,14,0.0,0.0,2.0,4.0,0.0,...,12.0,48.0,33.0,15.0,10.0,18.0,0.0,0.0,0.0,159.0
300,Montez Sweat,WAS,DL,16,16,1.0,15.0,6.0,2.0,0.0,...,9.0,45.0,28.0,17.0,12.0,20.0,0.0,15.0,1.0,155.5
202,J.J. Watt,HOU,DL,16,16,1.0,19.0,7.0,2.0,0.0,...,5.0,52.0,36.0,16.0,14.0,17.0,0.0,19.0,1.0,151.9
256,Chase Young *,WAS,DL,15,15,0.0,0.0,4.0,4.0,0.0,...,7.5,44.0,32.0,12.0,10.0,12.0,0.0,57.0,1.0,151.7
289,Leonard Williams,NYG,DL,16,12,0.0,0.0,1.0,0.0,0.0,...,11.5,57.0,29.0,28.0,14.0,30.0,0.0,0.0,0.0,146.0
364,Trey Hendrickson,NOR,DL,15,15,0.0,0.0,1.0,1.0,0.0,...,13.5,25.0,22.0,3.0,12.0,25.0,0.0,0.0,0.0,145.0
242,Stephon Tuitt,PIT,DL,15,15,0.0,0.0,3.0,2.0,0.0,...,11.0,45.0,33.0,12.0,10.0,25.0,0.0,0.0,0.0,144.0
185,DeForest Buckner +,IND,DL,15,14,0.0,0.0,3.0,2.0,0.0,...,9.5,58.0,37.0,21.0,10.0,26.0,1.0,0.0,0.0,142.0
250,Romeo Okwara,DET,DL,16,9,0.0,0.0,1.0,3.0,0.0,...,10.0,44.0,32.0,12.0,11.0,18.0,1.0,0.0,0.0,140.0
